In [2]:
import json
import csv

# Specify the path to your JSON file
json_file_path = "improve_model/openai_label_three_keywords.json"

# Specify the path to save the CSV file

csv_file_path = "improve_model/labels_3kw.csv"

# Load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Extract the data and write to CSV
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    header = ["topic_number", "Name"]
    max_keywords = 0  # Variable to track the maximum number of keywords
    for topic_number, topic_data in data.items():
        name = topic_data["Name"]
        openai_labels = topic_data["OpenAI_label"].split(",")  # Split the OpenAI_Label by comma
        num_keywords = len(openai_labels)  # Get the number of keywords for the current row
        if num_keywords > max_keywords:
            max_keywords = num_keywords
        row = [topic_number, name] + openai_labels  # Append all the keywords to the row
        writer.writerow(row)
    
    # Generate header for the keyword columns
    header += [f"OpenAI_Label_{i+1}" for i in range(max_keywords)]
    writer.writerow(header)

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# OpenAI label value counts

df = pd.read_csv("improve_model/labels_3kw.csv")

target_columns = ['OpenAI_Label_1', 'OpenAI_Label_2', 'OpenAI_Label_3']
# Combine all keyword columns into a single Series
all_keywords = pd.concat([df[column] for column in target_columns]).dropna()

# Compute the value counts for all keywords
keyword_counts = all_keywords.value_counts()

# Create a bar chart
plt.figure(figsize=(16, 6))  # Adjust figure size as per your preference
plt.bar(keyword_counts.index, keyword_counts.values)

# Set labels and title
plt.xlabel('Keyword')
plt.ylabel('Frequency')
plt.title('Combined Keyword Frequency')

# Rotate x-axis labels if needed
plt.xticks(rotation=90)

# Show the bar chart
plt.show() 

ParserError: Error tokenizing data. C error: Expected 5 fields in line 550, saw 6


In [2]:
import json
import pandas as pd
import csv

# Specify the path to your JSON file
json_file_path = "GBH_SUMMER_DEMO_DATA.json"

df = pd.read_json(json_file_path, orient='columns')

# Extract the information in the "articles" section and make keys in that dict the DataFrame's columns
# articles_df = pd.json_normalize(df['articles'], 'neighborhoods', 'position_section', 'tracts', 'author', 'body', 'content_id', 'hl1', 'hl2', 'pub_date', 'pub_name', 'link')
articles_df = pd.json_normalize(df['articles'])
# df = pd.concat([df, articles_df], axis=1)
# df.drop(columns=['articles'], inplace=True)

articles_df.describe

<bound method NDFrame.describe of                 neighborhoods position_section            tracts  \
0                    [Fenway]        Education          [010300]   
1     [Downtown, Beacon Hill]         Politics  [981700, 020302]   
2      [Dorchester, Mattapan]         Politics  [100900, 100700]   
3                  [Downtown]             News          [030302]   
4             [Jamaica Plain]        Education          [120400]   
...                       ...              ...               ...   
3812                      NaN              NaN               NaN   
3813                      NaN              NaN               NaN   
3814                      NaN              NaN               NaN   
3815                      NaN              NaN               NaN   
3816                      NaN              NaN               NaN   

                 author                                               body  \
0     Esteban Bustillos  Thomas White, a senior at Boston Latin School,

In [3]:
articles_df.dropna(axis=0)
articles_df.to_csv("geocoded_articles.csv")

In [4]:
# Helper code to merge two dataframes 
import pandas as pd

smaller_df = pd.read_csv("geocoded_articles.csv")

smaller_df.rename(columns={"body": "Body", "content_id": "Tagging"}, inplace=True)

larger_df = pd.read_csv("/projectnb/sparkgrp/mvoong/gbh-rss-articles/Articles Nov 2020 - March 2023.csv", usecols=['Tagging', 'Body'])

# larger_df.describe
merged_df = larger_df.merge(smaller_df[['Tagging', 'Body']], on='Tagging', how='left')[['Tagging', 'Body_x']]

merged_df = merged_df.dropna(subset=['Body_x'])
merged_df['Tagging'].iloc[240]
merged_df.to_csv('large_gbh_sample.csv')